# InterMineR Workshop Use Case



We are going to re-create the workflow we did using the web interface using the R API.

The basic steps are:  

1. Load the InterMine library and choose an InterMine to query.
2. Query 1: PAX6 + Pancreas: Fetch a list of genes that have medium or high expression in the pancreas and are in our PAX6 targets list
3. Query 2: Diabetes Genes: Fetch a list of genes that are associated with diabetes
4. Intersection: Find which genes are present in _both_ Query 1 and Query2.
5. GWAS: Compare the intersection of the previous query with results from GWAS studies.


### Getting started - Load InterMineR and choose an InterMine

Load the InterMine library. If it's not already installed, visit
https://bioconductor.org/packages/release/bioc/html/InterMineR.html and follow the instructions to install.


In [4]:
library(InterMineR)

We want to query human data - so let's look and see what InterMines are available: 

In [5]:
listMines()

BMAP 
               "https://bmap.jgi.doe.gov/bmapmine/" 
                                           BeanMine 
            "https://mines.legumeinfo.org/beanmine" 
                                         BovineMine 
           "http://genomes.missouri.edu/bovinemine" 
                                            CHOmine 
               "https://chomine.boku.ac.at/chomine" 
                                       ChickpeaMine 
        "https://mines.legumeinfo.org/chickpeamine" 
                                         CowpeaMine 
          "https://mines.legumeinfo.org/cowpeamine" 
                                            FlyMine 
                   "http://www.flymine.org/flymine" 
                                          GrapeMine 
         "http://urgi.versailles.inra.fr/GrapeMine" 
                                          HumanMine 
               "http://www.humanmine.org/humanmine" 
                                    HymenopteraMine 
     "http://hymenopteragenome.org/hymenopteramine" 
                                         IndigoMine 
              "http://www.cbrc.kaust.edu.sa/indigo" 
                                         LegumeMine 
          "https://mines.legumeinfo.org/legumemine" 
                                          MaizeMine 
"http://maizemine.rnet.missouri.edu:8080/maizemine" 
                                          MedicMine 
              "http://medicmine.jcvi.org/medicmine" 
                                          MitoMiner 
   "http://mitominer.mrc-mbu.cam.ac.uk/release-4.0" 
                                            ModMine 
        "http://intermine.modencode.org/release-33" 
                                          MouseMine 
               "http://www.mousemine.org/mousemine" 
                                            OakMine 
     "https://urgi.versailles.inra.fr/OakMine_PM1N" 
                                         PeanutMine 
          "https://mines.legumeinfo.org/peanutmine" 
                                          PhytoMine 
         "https://phytozome.jgi.doe.gov/phytomine/" 
                                           PlanMine 
              "http://planmine.mpi-cbg.de/planmine" 
                                            RatMine 
                   "http://ratmine.mcw.edu/ratmine" 
                                            RepetDB 
           "http://urgi.versailles.inra.fr/repetdb" 
                                            SoyMine 
             "https://mines.legumeinfo.org/soymine" 
                                         TargetMine 
   "https://targetmine.mizuguchilab.org/targetmine" 
                                          ThaleMine 
               "https://apps.araport.org/thalemine" 
                                          WheatMine 
        "https://urgi.versailles.inra.fr/WheatMine" 
                                           WormMine 
    "http://intermine.wormbase.org/tools/wormmine/" 
                                            XenMine 
                   "http://www.xenmine.org/xenmine" 
                                          YeastMine 
      "https://yeastmine.yeastgenome.org/yeastmine" 
                                      ZebrafishMine 
                         "http://zebrafishmine.org"

Okay, let's select HumanMine from the list:

In [8]:
humanMine <- listMines()["HumanMine"] #select humanmine
humanMine #print out the value to see what's inside

HumanMine 
"http://www.humanmine.org/humanmine"

Okay, now let's tell InterMineR that we want to use HumanMine for our queries.

**Important:** you'll need an API token for this part so you can access your HumanMine account. You can get your token by logging into [HumanMine](http://www.humanmine.org/) and going to the account details tab within MyMine. Cut and paste your token into the code below.

In [ ]:
im <- initInterMine(mine=humanMine, "YOUR TOKEN HERE")